In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# Prepare data
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms

trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
root = "/tmp"
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

batch_size = 1000

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

for X_train, y_train in train_loader:
    break
for X_val, y_val in test_loader:
    break
    
if True:
    X_train = X_train.double()
    X_val = X_val.double()
    
if torch.cuda.is_available():
    X_train = X_train.cuda()
    y_train = y_train.cuda()
    X_test = X_test.cuda()
    y_test = y_test.cuda()

In [11]:
# Build model
from tda.models import pytorch_lenet
lenet = pytorch_lenet.LeNet()
lenet.train_or_load()
if torch.cuda.is_available():
    lenet = lenet.cuda()

Loaded LeNet_pretrained.pth


In [12]:
# Load or train model
lenet.train_or_load(train_loader=[(X_train, y_train)], val_data=(X_val, y_val),
                    num_epochs=50)

Loaded LeNet_pretrained.pth


In [13]:
# Build architecture (tda pipeline terminology)
import tda.models.architectures.parser as parser
from imp import reload
reload(parser)

x = X_train[0]
lenet_arch = parser.model_to_architecture(lenet, name="mnist_lenet",
                                          x=x)

2020-05-18 14:15:10,938 - ConvLayer - INFO - <tda.models.layers.conv_layer.ConvLayer object at 0x7f7d61da26d0> received input with shape torch.Size([1, 1, 28, 28])
2020-05-18 14:15:10,940 - ConvLayer - INFO - <tda.models.layers.conv_layer.ConvLayer object at 0x7f7d61da2790> received input with shape torch.Size([1, 20, 12, 12])


In [14]:
from tda.graph import Graph

graph = Graph.from_architecture_and_data_point(lenet_arch, x)
for key in graph._edge_dict:
    layer_matrix = graph._edge_dict[key]
    print(layer_matrix.shape)

(11520, 784)
(2880, 11520)
(3200, 2880)
(800, 3200)
(500, 800)
(10, 500)


In [15]:
# Compute thresholds
from tda.thresholds import process_thresholds

class Dataset(object):
    def __init__(self, name, X_train, y_train, X_test, y_test):
        self.name = name
        self.train_dataset = list(zip(X_train, y_train))
        self.test_and_val_dataset = list(zip(X_test, y_test))
        
    def __str__(self):
        return self.name
    
# %debug
threshold = 0.01
raw_thresholds = "_".join([str(threshold)] * len(lenet_arch.layers))
dataset = Dataset("mnist", X_train, y_train, X_val, y_val)
thresholds = process_thresholds(architecture=lenet_arch,
                                dataset=dataset,
                                raw_thresholds=raw_thresholds,
                                dataset_size=10)

2020-05-18 14:15:15,361 - Cache - INFO - Cache root /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache/
2020-05-18 14:15:15,373 - Thresholds - INFO - Detected legacy format for thresholds
2020-05-18 14:15:15,374 - Thresholds - INFO - My received thresholds {(-1, 0): 0.01, (0, 1): 0.01, (1, 2): 0.01, (2, 3): 0.01, (3, 4): 0.01, (4, 5): 0.01}
2020-05-18 14:15:15,375 - Cache - INFO - Using cache file /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache//get_stats/architecture=mnist_lenet_e_0_dataset=mnist_dataset_size=10.cached for the call to get_stats
2020-05-18 14:15:15,758 - Thresholds - INFO - Link (-1, 0): threshold=960.4163540137355 (quantile 0.01)
2020-05-18 14:15:15,759 - Thresholds - INFO - Link (0, 1): threshold=10469.271519874757 (quantile 0.01)
2020-05-18 14:15:15,878 - Thresholds - INFO - Link (1, 2): threshold=37.45387282393914 (quantile 0.01)
2020-05-18 14:15:15,879 - Thresholds - INFO - Link (2, 3): threshold=2847.6980840866863 (quantile 0.01)
2020-05-18 1

In [16]:
# Build tda dataset (i.e activation graphs for clean and adversarial inputs)
from tda.protocol import get_protocolar_datasets

# %debug
lims = X_train.min(), X_train.max()
dataset_size = 200
all_epsilons = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
(train_clean, test_clean, train_adv,
 test_adv) = get_protocolar_datasets(dataset=dataset,
                                     succ_adv=True,
                                     dataset_size=dataset_size,
                                     noise=0., compute_graph=False,
                                     all_epsilons=all_epsilons,
                                     attack_type="FGSM_art",
                                     archi=lenet_arch,
                                     lims=lims)

2020-05-18 14:15:21,384 - C3PO - INFO - I will produce for you the protocolar datasets !
2020-05-18 14:15:21,391 - Cache - INFO - Using cache file /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache//get_sample_dataset/adv=False_archi=mnist_lenet_e_0_attack_type=FGSM_art_compute_graph=False_dataset=mnist_dataset_size=100_epsilon=0.0_lims=(tensor(-0.5000), tensor(0.5000))_noise=0.0_offset=0_succ_adv=True_train=False_transfered_attacks=False.cached for the call to get_sample_dataset
2020-05-18 14:15:21,409 - Cache - INFO - Using cache file /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache//get_sample_dataset/adv=False_archi=mnist_lenet_e_0_attack_type=FGSM_art_compute_graph=False_dataset=mnist_dataset_size=100_epsilon=0.0_lims=(tensor(-0.5000), tensor(0.5000))_noise=0.0_offset=100_succ_adv=True_train=False_transfered_attacks=False.cached for the call to get_sample_dataset
2020-05-18 14:15:21,426 - Cache - INFO - Using cache file /home/elvis/CODE/FORKED/TDA_for_adv_robus

In [ ]:
import tda.graph

if False:
    print("\nComputing activation graphs for clean examples")
    for ds in train_clean, test_clean:
        for l, line in enumerate(ds):
            print(".", end="")
            if line.graph is None:
                ds[l] = line = line._replace(
                    graph=tda.graph.Graph.from_architecture_and_data_point(
                        lenet_arch, line.x))
        assert 0
print("\nComputing activation graphs for adversarial examples", end="")
for ds in train_adv, test_adv:
    for eps in ds:
        if eps != 0.2: continue
        print("\neps=%.3f" % eps)
        for l, line in enumerate(ds[eps]):
            print(".", end="")
            if line.graph is None:
                ds[eps][l] = line._replace(
                graph=tda.graph.Graph.from_architecture_and_data_point(
                lenet_arch, line.x))
    break

In [ ]:
# Compute embeddings for test-set adversarial inputs
from tda.embeddings import get_embedding, EmbeddingType, KernelType, ThresholdStrategy
from joblib import delayed, Parallel


def embedding_getter(line):
    embedding = get_embedding(
        architecture=lenet_arch,
        embedding_type=EmbeddingType.PersistentDiagram,
        line=line, dataset=None, edges_to_keep=None,
        threshold_strategy=ThresholdStrategy.ActivationValue,
        thresholds=thresholds)
    print(".", end="")
    return embedding


n_jobs = 2
embeddings = {}
for eps in test_adv:
    print("\nComputing test adversarial embeddings for eps=%.3f" % eps)
    embeddings[eps] = Parallel(n_jobs=n_jobs)(
        delayed(embedding_getter)(line)
        for line in test_adv[eps])


Computing test adversarial embeddings for eps=0.010

Computing test adversarial embeddings for eps=0.050

Computing test adversarial embeddings for eps=0.100


In [ ]:
embeddings[0] = Parallel(n_jobs=n_jobs)(
        delayed(embedding_getter)(line)
        for line in test_clean)

In [ ]:
embeddings_test_adv = dict((eps, embeddings[eps]) for eps in embeddings if eps != 0)

In [ ]:
# Compute other adversarial examples
embeddings_train_adv = {}
for eps in train_adv:
    print("\nComputing train adversarial embeddings for eps=%.3f" % eps)
    embeddings_train_adv[eps] = Parallel(n_jobs=n_jobs)(
        delayed(embedding_getter)(line) for line in train_adv[eps])
print("\nComputing train clean embeddings")
embeddings_train_clean = Parallel(n_jobs=n_jobs)(
        delayed(embedding_getter)(line)
        for line in train_clean)
print("\nComputing test clean embeddings")
embeddings_test_clean = Parallel(n_jobs=n_jobs)(
        delayed(embedding_getter)(line)
        for line in test_clean)

In [ ]:
# The real deal: try to detect adversarial examples from normal examples
from tda.protocol import evaluate_embeddings

param_space = [{"M": 20, "sigma": sigma} for sigma in np.logspace(-3, 3, 7)]
kernel_type = KernelType.SlicedWasserstein
evaluation_results = evaluate_embeddings(embeddings_train_clean,
                                         embeddings_test_clean,
                                         embeddings_train_adv,
                                         embeddings_test_adv,
                                         kernel_type=kernel_type,
                                         param_space=param_space)

In [ ]:
# visualize embeddings
import matplotlib as mpl
import matplotlib.cm as cm

import seaborn as sns

cmap = cm.Blues_r

colors = {0: "b",
          0.1: "c",
          0.2: "m",
          0.3: "r"}
_, (ax1, ax3, ax4) = plt.subplots(1, 3, figsize=(15, 5))
for eps in embeddings:
    if not eps in [0., 0.1, 0.3]: continue
    color = colors[eps]
    for x in embeddings[eps]:
        birth, death = np.transpose(x)
        age = death - birth
        ax1.plot(birth, c=color)
        ax1.set_ylabel("birth")
        ax1.set_xlabel("points")
        # ax2.plot(death, c=color)
        # ax2.set_ylabel("death")
        # ax2.set_xlabel("points")
        ax3.plot(age, c=color)
        ax3.set_ylabel("age (death - birth)")
        ax3.set_xlabel("points")
        ax4.scatter(birth, death, c=color);
        ax4.set_xlabel("birth")
        ax4.set_ylabel("death")
plt.tight_layout()

In [ ]:
# Plot performance of the detector
import pandas as pd
import seaborn as sns

df = []
for key in ["supervised_metrics", "unsupervised_metrics"]:
    tmp = evaluation_results[key]
    if key == "unsupervised_metrics":
        sup = False
    else:
        sup = True
    for eps in tmp:
        df.append(dict(sup=sup, eps=eps, auc=tmp[eps]["auc"]["upper_bound"],
                       method="PersistentDiagram",
                       arch=lenet_arch.name))
df = pd.DataFrame(df)

In [ ]:
sns.pointplot(data=df, x="eps", y="auc", hue="sup");

In [ ]:
df

In [ ]:
layers

In [ ]:
lenet_arch.layers[-2].get_matrix()

In [ ]:
import scipy.sparse.linalg as slinalg

In [ ]:
mat = graph.get_adjacency_matrix()
mat

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(15, 8))
data = lenet_arch.layers[0].get_matrix()[-1].todense()[:2000].T
data = np.ma.masked_where(data == 0, data)
ax.matshow(data, cmap=plt.cm.RdBu);
ax.axis("off")

In [ ]:
graph._edge_dict.keys()

In [ ]:
lenet_arch.layers[4].get_matrix()

In [ ]:
from tda.graph import Graph

Graph.from_architecture_and_data_point(lenet_arch, X_train[0])

In [ ]:
graph._edge_dict

In [ ]:
from ripser.ripser import Rips

rips = Rips(maxdim=1)

In [ ]:
rips.fit_transform(graph.get_adjacency_matrix())

In [ ]:
graph.

In [ ]:
def get_edge_list(graph):
    """
    Generate the list of edges of the multipartite graph
    """
    shapes = graph._get_shapes()
    all_layer_indices = sorted(list(shapes.keys()))
    vertex_offset = [0] + list(np.cumsum([shapes[idx]
                                          for idx in all_layer_indices]))
    vertex_offset = vertex_offset[:-1]
    for source_layer, target_layer in graph._edge_dict:
        offset_source = vertex_offset[source_layer + 1]
        offset_target = vertex_offset[target_layer + 1]
        mat = graph._edge_dict[(source_layer, target_layer)]
        source_vertices = mat.col + offset_source
        target_vertices = mat.row + offset_target
        for edge, weight in zip(zip(source_vertices, target_vertices), mat.data):
            yield edge, weight

In [ ]:
timeit -n 1 graph.get_edge_list();

In [ ]:
timeit -n 1 list(get_edge_list(graph));

In [ ]:
del el

In [ ]:
8.08 / .777

In [ ]:
from tda.embeddings import persistent_diagrams
from tda import graph
from imp import reload

reload(persistent_diagrams)
reload(graph)
graph = graph.Graph.from_architecture_and_data_point(lenet_arch, X_train[0])


In [ ]:
toto = persistent_diagrams._prepare_edges_for_diagram_old(graph);
titi = persistent_diagrams._prepare_edges_for_diagram_fast(graph);

In [ ]:
titi = list(titi)

In [ ]:
titi[-1], toto[-1]

In [ ]:
for vertices, w in persistent_diagrams._prepare_edges_for_diagram_fast(graph):
    print(vertices, w)

In [ ]:
lenet_arch.layer_links

In [ ]:
import tda.embeddings.raw_graph as raw_graph

for layer_link in lenet_arch.layer_links[::-1]:
    active_indices = raw_graph.identify_active_indices([raw_graph.to_sparse_vector(
        line.graph._edge_dict[layer_link]) for line in train_clean])
    break

In [ ]:
line.graph._edge_dict[layer_link].shape

In [ ]:
a = train_clean[4].graph._edge_dict[layer_link].todense()
b = train_clean[11].graph._edge_dict[layer_link].todense()
c = train_adv[0.2][4].graph._edge_dict[layer_link].todense()
plt.matshow(a[:, :200])
plt.axis("off")
plt.matshow(b[:, :200])
plt.axis("off");
plt.matshow(c[:, :200])
plt.axis("off");

In [ ]:
from sklearn.random_projection import SparseRandomProjection

srp = SparseRandomProjection(random_state=0)

proj_mat = srp._make_random_matrix(100, 11520 * 784)

In [ ]:
plt.matshow(mean[:, :100])
plt.axis("off");

In [ ]:
plt.plot(proj_mat.dot(a.ravel().T))

In [ ]:
b[0]

In [ ]:
y_clean = np.array([line.y for line in train_clean])
graph_clean = [line.graph for line in train_clean]

In [ ]:
indices = np.where(y_clean == 6)[0].build_matrix()

In [ ]:
mean = np.mean([graph_clean[i]._edge_dict[layer_link].todense() for i in indices], axis=0)

In [ ]:
a = lenet_arch.layers[2].build_matrix(old=True).todense()
b = lenet_arch.layers[2].build_matrix(old=False).todense()

import numpy as np
a - b

In [ ]:
timeit lenet_arch.layers[2].build_matrix(old=False)

In [ ]:
timeit lenet_arch.layers[2].build_matrix(old=True)

In [6]:
import sys
sys.path.append("../universal-attacks/")

from imp import reload
import torch
import universal_attacks.models as models
reload(models)

# load pretrained model and prepare dataset fetcher
model, ds_fetcher, class_names = models.get_cifar100(
    model_root="pytorch_data")
class_indices = dict(zip(class_names.values(), class_names.keys()))

# turn off dropout
model.eval()

# fetch data
for X_train, y_train in ds_fetcher(batch_size=1000, train=True, val=False):
    break
for X_test, y_test in ds_fetcher(batch_size=50000, train=False, val=True):
    break
    
# save mem
if False:
  model = model.float()
  X_train = X_train.float()
  y_train = y_train.float()
  X_test = X_test.float()
  y_test = y_test.float()

# transfer data to GPU
if torch.cuda.is_available():
    X_train = X_train.cuda()
    y_train = y_train.cuda()
    X_test = X_test.cuda()
    y_test = y_test.cuda()

Building and initializing cifar100 parameters
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padd

Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified


In [7]:
reload(parser)
arch = parser.model_to_architecture(model, x=X_train[0])

2020-05-17 08:55:18,670 - ConvLayer - INFO - <tda.models.layers.conv_layer.ConvLayer object at 0x7f778f53ecd0> received input with shape torch.Size([1, 3, 32, 32])
<tda.models.layers.conv_layer.ConvLayer object at 0x7f778f53ecd0> received input with shape torch.Size([1, 3, 32, 32])
2020-05-17 08:55:18,673 - ConvLayer - INFO - <tda.models.layers.conv_layer.ConvLayer object at 0x7f778e8e9cd0> received input with shape torch.Size([1, 128, 32, 32])
<tda.models.layers.conv_layer.ConvLayer object at 0x7f778e8e9cd0> received input with shape torch.Size([1, 128, 32, 32])
2020-05-17 08:55:18,688 - ConvLayer - INFO - <tda.models.layers.conv_layer.ConvLayer object at 0x7f778e8e92d0> received input with shape torch.Size([1, 128, 16, 16])
<tda.models.layers.conv_layer.ConvLayer object at 0x7f778e8e92d0> received input with shape torch.Size([1, 128, 16, 16])
2020-05-17 08:55:18,694 - ConvLayer - INFO - <tda.models.layers.conv_layer.ConvLayer object at 0x7f778e8e95d0> received input with shape torch.

In [ ]:
from tda.graph import Graph

graph = Graph.from_architecture_and_data_point(arch, X_train[0])
for key in graph._edge_dict:
    layer_matrix = graph._edge_dict[key]
    print(layer_matrix.shape)

In [ ]:
for c in lenet.named_children():
    print(c)

In [ ]:
def _unroll(model):
    for child in model.children():
        if len(list(child.children())):
            for grandchild in _unroll(child):
                yield grandchild
        else:
            yield child

for c in _unroll(model.features):
    print(c)

In [ ]:
def lprint(a):
    if isinstance(a, list):
        for i in a:
            yield from lprint(i)
    else:
        yield a

a = [[1, [2, 3], 4], [5, 6, [7, 8, [9]]]]
for i in lprint(b):
    print(i)

In [ ]:
model.n